In [1]:
%load_ext autoreload
%autoreload 2

In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import math
import glob
import tqdm
from scipy.stats import pearsonr as pcorr
import itertools
import re
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from ipywidgets import interact
# from src.features import metric_exploration

In [3]:
PATH_ROOT = "/home/shaul/workspace/GitHub/SOTA/"

In [4]:
cd {PATH_ROOT}

/home/shaul/workspace/GitHub/SOTA


In [5]:
df = pd.read_csv('/home/shaul/workspace/GitHub/SOTA/data/full_DS/full_metrics.csv', index_col= 0)
with open('/home/shaul/workspace/GitHub/SOTA/data/other/ba_all.txt','r+') as f:
    list_ba = f.read().splitlines() 

In [6]:
# # Need to change 'is_radical' and 'is_centrist' into one category
# df['radical_or_centralist'] = ['radical'if row['is_radical'] ==True else "centralist"  if row['is_centralist'] == True else "neither" for index,row in df.iterrows()]
# df.drop(columns=['is_radical','is_centralist'], inplace=True)

In [7]:
non_metric_columns = ['text1','text2','label','dataset','random','duration','total_seconds','pair_id','reduced_label','annotator','radical','radical_random','radical_non_random','radical_or_centralist','num_labels','bad_annotator']
categories = ['dataset', 'random','radical_or_centralist']

In [8]:
df.columns

Index([&#39;annotator&#39;, &#39;text1&#39;, &#39;text2&#39;, &#39;label&#39;, &#39;dataset&#39;, &#39;random&#39;, &#39;duration&#39;,
       &#39;total_seconds&#39;, &#39;pair_id&#39;, &#39;reduced_label&#39;, &#39;bleu&#39;, &#39;bleu1&#39;,
       &#39;glove_cosine&#39;, &#39;fasttext_cosine&#39;, &#39;BertScore&#39;, &#39;chrfScore&#39;,
       &#39;POS Dist score&#39;, &#39;1-gram_overlap&#39;, &#39;ROUGE-1&#39;, &#39;ROUGE-2&#39;, &#39;ROUGE-l&#39;,
       &#39;radical&#39;, &#39;radical_random&#39;, &#39;radical_non_random&#39;, &#39;num_labels&#39;,
       &#39;bad_annotator&#39;, &#39;L2_score&#39;, &#39;WMD&#39;, &#39;radical_or_centralist&#39;],
      dtype=&#39;object&#39;)

In [10]:
from src import metric_exploration
test = metric_exploration.Metrics_Corr(df,non_metric_columns,categories)

In [11]:
example = test.compare_correlations(list_ba)

In [12]:
example['label_by_random']

,bleu,bleu1,glove_cosine,fasttext_cosine,BertScore,chrfScore,POS Dist score,1-gram_overlap,ROUGE-1,ROUGE-2,ROUGE-l,L2_score,WMD
0,0.006192,0.036388,-0.025129,-0.038641,0.029053,0.029845,0.002567,0.023812,0.032359,0.026236,0.036233,-0.040784,-0.027279
1,-0.017561,-0.045880,0.038510,0.041842,0.012594,-0.040251,-0.008199,-0.043527,-0.040694,-0.028073,-0.031789,0.028917,0.002130


### Look at the Non-Linear and Linear Models

In [13]:
from src import metric_exploration

test = metric_exploration.Metrics_Models(df,non_metric_columns,categories)

In [14]:
scores, fi_values = test.run_model(model_type = "RF")

In [42]:
scores.keys()

dict_keys([&#39;dataset&#39;, &#39;random&#39;, &#39;radical_or_centralist&#39;, &#39;RF_label_combined&#39;, &#39;RF_label_reduced_combined&#39;])

In [67]:
for categ in categories:
    @interact
    def plot_fi_dataset(key_v = fi_values[categ].keys()):
        plt.figure(figsize=(10,5))
        chart = sns.barplot(list(fi_values[categ][key_v].feature), fi_values[categ][key_v].importance);
        chart.set_xticklabels(chart.get_xticklabels(),rotation=45);
        chart.set_title(categ)
        plt.show();

others = [x for x in fi_values.keys() if x not in categories]
@interact
def plot_fi(key_v = others):
    plt.figure(figsize=(10,5))
    chart = sns.barplot(list(fi_values[key_v].feature), fi_values[key_v].importance);
    chart.set_xticklabels(chart.get_xticklabels(),rotation=45);
    chart.set_title(key_v)
    plt.show();

interactive(children=(Dropdown(description=&#39;key_v&#39;, options=(&#39;fi_label_by_dataset_bible_human&#39;, &#39;fi_reduced_la…

interactive(children=(Dropdown(description=&#39;key_v&#39;, options=(&#39;fi_label_by_random_0&#39;, &#39;fi_reduced_label_by_rand…

interactive(children=(Dropdown(description=&#39;key_v&#39;, options=(&#39;fi_label_by_radical_or_centralist_centralist&#39;, &#39;…

interactive(children=(Dropdown(description=&#39;key_v&#39;, options=(&#39;fi_label_combined&#39;, &#39;fi_reduced_label_combined&#39;)…